## Make finegrained labels

In [27]:
from ast import literal_eval
import html
import re
import pandas as pd


def clean_text(text):

    inp = re.sub(r'(u/.*? |@.*?( |$))', "<user> ", text)
    inp = re.sub(r'\u2019', "'", inp)
    inp = re.sub(r'[“”]', '"', inp)
    inp = re.sub(r'\\u[a-f0-9]{4}', "'", inp)
    inp = re.sub(r'\(https://.*\)', "(URL)", inp)

    inp = re.sub(r'[\s]{2,}', " ", inp)
    inp = html.unescape(inp).encode('ascii', errors='ignore').decode()
        
    return inp

def process_alpaca(text, label, instruction=None):
#     instruction = defs
    
    out = {'instruction': [],
           'input': [],
           'output': []
          
          }
    
    for i in range(len(text)):
        
        outp = label[i]
        
        
        inp = clean_text(text[i].strip())
        
        if type(instruction) is str:
            out['instruction'].append(instruction)
        elif type(instruction) is list:
            out['instruction'].append(instruction[i])
            
        out['input'].append(inp)
        out['output'].append(outp)
    return out

def openprompt_to_prompt(test_enc):
    instructions = []
    test_prompt = []
    
    output = []
    
    for i in range(len(test_enc)):
        ins = '{target} is defined as {target_def}'.format(target=test_enc[i].meta['target'],
                                                                         target_def=test_enc[i].meta['definition'])
        if test_enc[i].meta['target_groups'] is not None:
            ins += 'Target groups include {0}'.format(', '.join(test_enc[i].meta['target_groups']))
        instructions.append(ins)
        test_prompt.append('Post: {text} Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(
            target=test_enc[i].meta['target'], 
            target_def=test_enc[i].meta['definition'],
            text=test_enc[i].text_a))
        
        output.append(('Yes' if test_enc[i].label == 2 else ('Unclear' if test_enc[i].label == 1 else 'No')))
    return process_alpaca(test_prompt, output, instructions)

In [48]:
full = pd.read_csv('data/full_dataset.csv')
negative = pd.read_csv('data/negative_for_dataset.csv')

In [49]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")


full['tokens'] = [len(enc.encode(x)) for x in full['text'].to_list()]
negative['tokens'] = [len(enc.encode(x)) for x in negative['text'].to_list()]

In [53]:
full_temp = full[full['tokens'] < 400]
to_sample = len(full) - len(full_temp)

In [56]:

full = pd.concat([full_temp, negative[(~negative['reddit_id'].isin(full_temp['reddit_id'])) & (negative['tokens'] < 400)].sample(n=to_sample)])

In [57]:
full, to_sample

(      Unnamed: 0.1  Unnamed: 0  \
 0                0           0   
 1                1           1   
 2                2           2   
 3                3           3   
 4                4           4   
 ...            ...         ...   
 3072       2664156     2099983   
 2119      12612491    11369425   
 965       21775426    19221085   
 3998      11818772    10687737   
 3868      11138731    10118096   
 
                                                    text  original_id  \
 0     Did he kill your sex life? Women are objects s...       1006.0   
 1     Don’t support our public schools because child...       1018.0   
 2     Tricky Dick. I hope you get hit by a car the n...       1022.0   
 3     Canadians are physically and mentally inferior...       1026.0   
 4     Is that cat waving a Nazi banner? People with ...       1031.0   
 ...                                                 ...          ...   
 3072  Are you talking about Vital Farms eggs? I love...          N

In [ ]:
full['notes'].isna().sum(), len(full)

In [ ]:
len(full)

In [ ]:

legal_defs = pd.read_csv('data/legal definitions.csv')
legal_labels = legal_defs['name'].to_list()
defs = pd.read_csv('data/definitions.csv')

In [ ]:
groups = {
         'Race (colour, creed)': ['actual and perceived race', 'colour', 'creed', 'race', 'race or perceived race'],
         'National or ethnic origin (nationality, ethnicity, ancestry)': ['ancestry', 'ethnic origin', 'ethnicity', 'place of origin', 'national origin', 'national or ethnic origin', 'nationality', 'indigenous identity'],
         'Political belief/association': ['political association', 'political belief'],
         'Sex/Gender identity or expression': ['gender', 'gender identity', 'gender identity and expression', 'gender identity or expression', 'sex', 'sex/gender'],
         'Religion/Religious beliefs': ['religion', 'religious affiliation','religious beliefs' ],
         'Sexual orientation': ['sexual orientation'],
         'Social condition': ['social condition'],
         'Immigration status': ['immigration status'],
         'Source of income': ['source of income'],
         'Age': ['age'],
         'Physical or mental disability': ['physical or mental disability', 'mental or physical disability', 'physical disability', 'mental disability', 'disability', 'pregnancy or disability'],
         'Family affiliation': ['family affiliation', 'caste'],
         'Conviction that is subject to a pardon or record suspension': ['conviction that is subject to a pardon or record suspension'],
         'Receipt of public assistance': ['receipt of public assistance'],
         'Serious disease': ['serious disease'],
         'Family status': ['family status'],
        'Pregnancy': ['pregnancy'],
    'Victims of a major violent event and their families/kin': ['victims of a major violent event and their kin', 'victims of a major event and their families'],
    'Veteran status': ['veteran status'],
    'Marital status': ['marital status']
}

groups = {x: k for k, v in groups.items() for x in v}

In [ ]:
groups

In [ ]:
# g = set([x.strip().lower() for y in legal_defs['protected_groups'].to_list() for x in y.split(',')])

# temp1 = []

# for grps in list(g):
#     if grps not in groups.keys():
#         print(grps)
        

    

In [ ]:
legal_defs

In [ ]:
gr = []

for i in legal_defs['protected_groups'].to_list():
    temp = [groups[x.strip().lower()] for x in i.split(',')]
    
    gr.append(list(set(temp)))

In [ ]:
legal_defs['protected_groups_cleaned'] = gr

In [ ]:
finegrained = []

for i in range(len(full)):
    label = full.iloc[i]['label']
    
    if label == 'Violates' and type(full.iloc[i]['target_group']) is list:
        temp = []
        for g in full.iloc[i]['target_group']:
            for j in range(2,len(legal_defs)):
                if g in legal_defs.iloc[j]['protected_groups_cleaned']:
                    temp.append(legal_labels[j])
        
        if full.iloc[i]['cc_318']:
            temp.append('CC_318')
        if full.iloc[i]['cc_319']:
            temp.append('CC_319')
        print(temp)
        finegrained.append(temp)
    else:
        finegrained.append([])

In [ ]:
full['finegrained'] = [([] if pd.isna(x) else (literal_eval(x) if type(x) is str else x)) for x in full['finegrained'].to_list()]

In [ ]:

full['finegrained'] = [(list(set(finegrained[i] + full['finegrained'].iloc[i])) if full.iloc[i]['label'] == 'Violates' else None) for i in range(len(full))]

In [ ]:
full[full['id_x'] == 1006]

In [ ]:
full[full['source'] == 'edited']['label'].value_counts()

In [ ]:
full.to_csv('data/full_dataset.csv')

In [ ]:
prompt = ['{target} states "{target_def}" Protected groups include {targets}'.format(
    target=legal_defs.iloc[i]['promptName'],
    target_def=legal_defs.iloc[i]['definition'],
    targets=legal_defs.iloc[i]['protected_groups_cleaned']
) for i in range(len(legal_defs))]

legal_defs['prompt'] = prompt

prompt = '\n'.join(prompt)


In [ ]:
# mapping = ['Violates': 'Yes', ]
from datasets import Dataset

from pathlib import Path

def process_dataset_nonotes(full, legal_defs, name):
    for i in range(len(legal_defs)):

        inp = ['Post: {text} Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(
                target=legal_defs.iloc[i].promptName, 
                text=x) for x in full['text'].to_list()]

        outp = [('Yes' if (full.iloc[j]['label'] == 'Violates' and legal_labels[i] in full.iloc[j]['finegrained']) else ('Unclear' if full.iloc[j]['label'] == 'Meaning unclear' else 'No')) for j in range(len(full))]

    #     for j in range(len(full)):
    #         if type(full.iloc[j]['notes']) is list and len(full.iloc[j]['notes']) > 0:

        sequences = process_alpaca(text=inp, label=outp, instruction=legal_defs.iloc[i]['prompt'])

        Dataset.from_dict(sequences).to_json('{1}/singletarget_nonotes_{0}.json'.format(legal_labels[i], name))
        
        
    prompt = '\n'.join(legal_defs['prompt'].to_list())
    inp = ['Post: {text} Q: Is this post a violation of any of the above policies? Please answer Yes/No and which policies. A: '.format(
        target=legal_defs.iloc[i].promptName, 
        text=x) for x in full['text'].to_list()]

    outp = [('Yes' if (full.iloc[j]['label'] == 'Violates' and legal_labels[i] in full.iloc[j]['finegrained']) else ('Unclear' if full.iloc[j]['label'] == 'Meaning unclear' else 'No')) for j in range(len(full))]

    #     for j in range(len(full)):
    #         if type(full.iloc[j]['notes']) is list and len(full.iloc[j]['notes']) > 0:

    sequences = process_alpaca(text=inp, label=outp, instruction=prompt)

    Dataset.from_dict(sequences).to_json('{1}/multitarget_nonotes.json'.format(legal_labels[i], name))
    
def process_dataset_notes(full, legal_defs, name):
    for i in range(len(legal_defs)):
    
        inputs = []

        outputs = []

        for j in range(len(full)):

            inp = 'Post: {text} Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(
                target=legal_defs.iloc[i].promptName, 
                text=full.iloc[j]['text'])

            outp = ('Yes' if (full.iloc[j]['label'] == 'Violates' and legal_labels[i] in full.iloc[j]['finegrained']) else 
                 ('Unclear' if full.iloc[j]['label'] == 'Meaning unclear' else 'No'))
            if type(full.iloc[j]['notes']) is list and len(full.iloc[j]['notes']) > 0:
                for note in full.iloc[j]['notes']:
                    inputs.append(inp)
                    outputs.append(outp + ', {0}'.format(note))
            else:
                inputs.append(inp)
                outputs.append(outp)
        sequences = process_alpaca(text=inputs, label=outputs, instruction=legal_defs.iloc[i]['prompt'])

        Dataset.from_dict(sequences).to_json('{1}/singletarget_notes_{0}.json'.format(legal_labels[i], name))
        
    prompt = '\n'.join(legal_defs['prompt'].to_list())

    for j in range(len(full)):

        inp = 'Post: {text} Q: Is this post a violation of any of the above policies? Please answer Yes/No and which policies. A: '.format(
            target=legal_defs.iloc[i].promptName, 
            text=full.iloc[j]['text'])

        outp = ('Yes' if (full.iloc[j]['label'] == 'Violates' and legal_labels[i] in full.iloc[j]['finegrained']) else 
             ('Unclear' if full.iloc[j]['label'] == 'Meaning unclear' else 'No'))
        if type(full.iloc[j]['notes']) is list and len(full.iloc[j]['notes']) > 0:
            for note in full.iloc[j]['notes']:
                inputs.append(inp)
                outputs.append(outp + ', {0}'.format(note))
        else:
            inputs.append(inp)
            outputs.append(outp)
    sequences = process_alpaca(text=inputs, label=outputs, instruction=prompt)

    Dataset.from_dict(sequences).to_json('{1}/multitarget_notes.json'.format(legal_labels[i], name))
    

    
def process_dataset(full, legal_defs, name):
    
    Path(name).mkdir(parents=True, exist_ok=True)
    print('directory created (if it didn\'t already exist)')
    print('formatting without notes')
    process_dataset_nonotes(full, legal_defs, name)
    print('formatting with notes')
    process_dataset_notes(full, legal_defs, name)

In [ ]:
process_dataset(full, legal_defs, 'zeroshot')

In [ ]:
train = full.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.6))

In [ ]:
test = full[~full.index.isin(train.index)]

In [ ]:
test['label'].value_counts()

In [ ]:
process_dataset(train, legal_defs, 'train')

In [ ]:
process_dataset(test, legal_defs, 'test')

In [ ]:
train.to_csv('data/legal_train.csv')
test.to_csv('data/legal_test.csv')

In [ ]:
train['text'] = [clean_text(x) for x in train['text'].to_list()]

In [ ]:
test['text'] = [clean_text(x) for x in test['text'].to_list()]